In [1]:
import numpy as np
import pandas as pd
import TIRCP_functions
import shared_utils
import altair as alt
import altair_saver
from shared_utils import altair_utils 
from statistics import mode
from collections import Counter

pd.options.display.max_rows = 100
pd.set_option('display.max_colwidth', None)

pd.options.display.float_format = "{:0f}".format
pd.options.display.float_format = '{:,}'.format

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df = TIRCP_functions.tableau()

In [3]:
df.columns

Index(['Award_Year', 'Project_#', 'Local_Agency', 'Vendor_ID_#',
       'Project_Title', 'District', 'County', 'Key_Project_Elements',
       'Master_Agreement_Number', 'Master_Agreement_Expiration_Date',
       'Project_Manager', 'Regional_Coordinator',
       'Technical_Assistance-CALTP_(Y/N)', 'Technical_Assistance-Fleet_(Y/N)',
       'Technical_Assistance-Network_Integration_(Y/N)',
       'Technical_Assistance-Priority_Population_(Y/N)', 'Total_Project_Cost',
       'TIRCP_Amount', 'Allocated_Amount', 'Unallocated_amt_project_sheet',
       'Percentge_Allocated', 'Expended_Amount', 'Other_Funds_Involved',
       'Award_Cycle', 'Local_Agency_Address', 'Local_Agency_City',
       'Local_Agency_Zip', 'Local_Agency_Contact', 'Local_Agency_Email',
       'Local_Agency_Phone_Number', 'Comments/Additional_Contacts', 'PPNO',
       'Expended_Percent', 'Allocated_Percent', 'Expended_Percent_Group',
       'Progress', 'Project_Category'],
      dtype='object')

## 1. What % of projects are on time?
* 58% of projects have NO expenditure. 
* 28% are on track. 

In [4]:
df1 = df.groupby(['Progress']).agg({'PPNO':'count'}).rename(columns = {'PPNO':'Count_of_Projects', 'TIRCP_Amount':'Sum_of_TIRCP'}) 
df1['Percentage_of_Projects'] = ((100* df1['Count_of_Projects']/(df1['Count_of_Projects']).sum())).round(0)
sort1 = ['No Expenditures', 'Behind','On Track','Ahead'] 
df1 = df1.loc[sort1].reset_index().sort_values('Percentage_of_Projects')
df1

,Progress,Count_of_Projects,Percentage_of_Projects
1,Behind,5,7.0
3,Ahead,5,7.0
2,On Track,21,28.0
0,No Expenditures,43,58.0


In [5]:
chart_progress = TIRCP_functions.basic_bar_chart(df1, 'Progress','Percentage_of_Projects', 'Progress') 
chart_progress

alt.Chart(...)

## 2. Progress of bigger projects.
* Most projects are medium (36), followed by large (19), small (16), and $0 in TIRCP amount (3). 
* Medium, large, and small defined by percentiles. 
* Most projects are "medium" with no expenditures. 
* No small projects are on track.

In [6]:
df.groupby('Project_Category').agg({'PPNO':'count'})

,PPNO
Project_Category,
$0 recorded for TIRCP,3
Large,19
Medium,36
Small,16


In [7]:
df2 = df.groupby(['Project_Category', 'Progress']).agg({'PPNO':'count', 'TIRCP_Amount':'sum'}).rename(columns = {'PPNO':'Count_of_Projects', 'TIRCP_Amount':'Sum_of_TIRCP'}) 
df2['Percentage_of_Projects'] = ((100* df2['Count_of_Projects']/(df2['Count_of_Projects']).sum())).round(0)
sort2 = ['Small', 'Medium','Large'] 
df2 = df2.loc[sort2].reset_index()
df2

,Project_Category,Progress,Count_of_Projects,Sum_of_TIRCP,Percentage_of_Projects
0,Small,Behind,1,"1,675,000.0",1.0
1,Small,No Expenditures,11,"49,645,000.0",15.0
2,Small,On Track,4,"15,301,000.0",5.0
3,Medium,Ahead,3,"79,617,000.0",4.0
4,Medium,Behind,2,"19,999,000.0",3.0
5,Medium,No Expenditures,20,"404,971,000.0",27.0
6,Medium,On Track,11,"210,604,000.0",15.0
7,Large,Ahead,2,"580,840,000.0",3.0
8,Large,Behind,2,"469,209,000.0",3.0
9,Large,No Expenditures,10,"2,634,906,000.0",14.0


In [8]:
chart_project_size4 = TIRCP_functions.basic_bar_chart(df2, 'Progress','Percentage_of_Projects', 'Project_Category') 
chart_project_size4

alt.Chart(...)

In [9]:
chart_project_size2 = TIRCP_functions.basic_bar_chart(df2, 'Project_Category','Count_of_Projects', 'Progress') 
chart_project_size2

alt.Chart(...)

## 3. Which organizations received the most $?

In [10]:
df3 = df.groupby(['Local_Agency']).agg({'PPNO':'count', 'TIRCP_Amount':'sum'}).reset_index().rename(columns = {'PPNO':'Count_of_Projects','TIRCP_Amount':'TIRCP_Received'}) 
df3= df3.sort_values('TIRCP_Received').tail(5)
df3

,Local_Agency,Count_of_Projects,TIRCP_Received
3,Bay Area Rapid Transit District (BART),2,"425,700,000.0"
27,San Joaquin Joint Powers Authority (SJJPA) & San Joaquin Regional Rail Commission (SJRRC),1,"500,500,000.0"
32,Santa Clara Valley Transportation Authority,2,"750,000,000.0"
37,Southern California Regional Rail Authority,2,"916,889,000.0"
15,Los Angeles County Metropolitan Transportation Authority,4,"1,236,202,000.0"


In [11]:
org_most_money = TIRCP_functions.basic_bar_chart(df3, 'Local_Agency','TIRCP_Received','Local_Agency') 
org_most_money

alt.Chart(...)

## 4. Project Type - seeing how many goals each project hits.
* [Picking out keywords](https://www.geeksforgeeks.org/python-program-for-most-frequent-word-in-strings-list/)
* [TIRCP](https://calsta.ca.gov/subject-areas/transit-intercity-rail-capital-prog)
* [2018 Projects](https://calsta.ca.gov/-/media/calsta-media/documents/2018-tircp-detailed-project-award-announcement.pdf)

<b> Goals of TIRCP </b>
1. Modernize California's transit. 
2. Reduce emissions of greenhouse gases
3. Expand and improve transit service to increase ridership
4. Integrate the rail service of the state’s various rail operations, including integration with the high‐speed rail system
5. Improve transit safety



In [12]:
#convert all key project elements to list to read through
project_elements_list = df['Key_Project_Elements'].unique().tolist()

In [13]:
#1
ENVIRONMENT = ['electric','zero-emission', 'Reduce Emissions', 
               'battery','hydrogen fuel-cell',
              'clean','emissions','emission','greenhouse gas']
#2
EXPANSION = ['passengers', 'increase use of transit', 'increase transit service', 'customer',
             'expand service','service expansion', 'expansion','increase frequencies', 
             'customer focused', 'frequency', 'parking spaces','on-time performance',
           'capacity-increasing', 'ridership','mobility','limited access','bus','rail', 'light rail vehicles', 'trolley vehicles', 'locomotives', 
            'vanpool', 'electrification', 'streetcar', 'LRV', 'low-floor rail vehicles',
           'Zero Emission Multiple Unit (ZEMU) train','ZEMU', 'buses', 'powered automated people mover (APM)'
           ]

#3
SAFETY = ['safety','safe','overcrowding']

#4
MODERNIZE = ['install','communications upgrade', 'construct tracks', 'infrastructure', 
               'extension', 'lengthens platforms','modernize', 'expanding', 'network integration']

#5
INTEGRATION = ['connecting','linking', 'seamless', 'service extension','connections', 'extend',
            'transit-only', 'expand services', 'extend','extended', 'expansion','expands','link','track']


In [14]:
def categorize_project_descriptions(row):
    """
    This function takes a individual type of work description (row of a dataframe)
    and returns a dummy flag of 1 if it finds keyword present in
    project categories (active transportation, transit, bridge, etc).
    A description can contain multiple keywords across categories.
    """
    # Make lowercase
    description = row.Key_Project_Elements.lower()
    
    #Flagging columns. If a project doesn't have any of the keywords, flag as 0. If it does, flag as 1.
    environment = 0
    expansion = 0
    safety = 0
    modernize = 0
    integration = 0
    
    if any(word in description for word in ENVIRONMENT):
        environment = 1
    if any(word in description for word in EXPANSION):
        expansion = 1
    if any(word in description for word in SAFETY):
        safety = 1
    if any(word in description for word in MODERNIZE):
        modernize = 1
    if any(word in description for word in INTEGRATION):
        integration = 1
   # Create new cols out of our categories.
    return pd.Series(
        [environment, expansion, safety, modernize, integration], 
        index=['environment', 'expansion', 'safety', 'modernize','integration']
    )

In [15]:
#New DF with the environment/expansion/etc flagged.
project_categories = df.apply(categorize_project_descriptions, axis=1)

In [16]:
#Concating the 2 dataframes together, original and the one just created
keyword_df = pd.concat([df, project_categories], axis=1)

### Most projects hit 2 categories, followed by 1. 

In [17]:
#List of all our new cols & get value_counts for how many projects hit 1 or 2 or etc goals.
project_cols = list(project_categories.columns)

keyword_df = keyword_df.assign(
    project_categories = keyword_df[project_cols].sum(axis=1)
)

keyword_df.project_categories.value_counts()

2    25
1    19
3    19
4     6
0     4
5     1
Name: project_categories, dtype: int64

In [18]:
keyword_df2 = keyword_df[['PPNO','Award_Year', 'TIRCP_Amount','Key_Project_Elements', 'environment', 'expansion', 'safety', 'modernize','integration','project_categories']]

In [19]:
keyword_df2['TIRCP_Amount_Mean'] = keyword_df2['TIRCP_Amount'] 

/tmp/ipykernel_2498/2255519327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
keyword_df2 = keyword_df2.groupby(['environment', 'expansion', 'safety',
                     'modernize','integration','project_categories']).agg({'PPNO':'count','TIRCP_Amount':'sum',
                                                                          'TIRCP_Amount_Mean':'mean'}).reset_index()


### Looking at the top 5 projects by count of projects total & goals

In [21]:
keyword_projects_total = keyword_df2.sort_values('PPNO').tail(5).rename(columns = {'PPNO':'Total_Projects'}) 
keyword_projects_total

,environment,expansion,safety,modernize,integration,project_categories,Total_Projects,TIRCP_Amount,TIRCP_Amount_Mean
12,1,1,0,1,1,4,6,"347,583,000.0","57,930,500.0"
9,1,1,0,0,0,2,8,"233,855,000.0","29,231,875.0"
10,1,1,0,0,1,3,10,"183,496,000.0","18,349,600.0"
4,0,1,0,0,1,2,13,"1,208,974,000.0","92,998,000.0"
3,0,1,0,0,0,1,15,"616,957,000.0","41,130,466.666666664"


In [22]:
Categories1 = ['Environment, expansion, modernize, & integration', 
               'Environment & expansion', 'Environment, expansion & integration', 
               'Expansion & Integration', 'Expansion Only']

In [23]:
keyword_projects_total['Project_Goals'] = Categories1

In [24]:
Project_goals_total_projects = TIRCP_functions.basic_bar_chart(keyword_projects_total, 'Project_Goals','Total_Projects','Project_Goals') 
Project_goals_total_projects

alt.Chart(...)

### Looking at the top 5 projects by TIRCP total & goals

In [25]:
keyword_df3 = keyword_df2.sort_values('TIRCP_Amount').tail(5)

In [26]:
Categories2 = ['Expansion & Modernize', 'Expansion Only', 'Integration Only', 'Expansion & Integration', 'Expansion, Modernize & Integration']

In [27]:
keyword_df3['Project_Goals'] = Categories2
keyword_df3

,environment,expansion,safety,modernize,integration,project_categories,PPNO,TIRCP_Amount,TIRCP_Amount_Mean,Project_Goals
5,0,1,0,1,0,2,1,"500,500,000.0","500,500,000.0",Expansion & Modernize
3,0,1,0,0,0,1,15,"616,957,000.0","41,130,466.666666664",Expansion Only
1,0,0,0,0,1,1,4,"833,099,000.0","208,274,750.0",Integration Only
4,0,1,0,0,1,2,13,"1,208,974,000.0","92,998,000.0",Expansion & Integration
6,0,1,0,1,1,3,4,"1,589,839,000.0","397,459,750.0","Expansion, Modernize & Integration"


In [28]:
Project_Goals_TIRCP = TIRCP_functions.basic_bar_chart(keyword_df3, 'Project_Goals','TIRCP_Amount','Project_Goals') 
Project_Goals_TIRCP

alt.Chart(...)

### Simpler view to look at projects that meet 1/2/etc of the goals
* Projects can meet multiple goals.

In [29]:
value_count_list = keyword_df[['environment', 'expansion', 'safety', 'modernize','integration']]

In [30]:
for c in value_count_list.columns:
    print (value_count_list[c].value_counts())

0    43
1    31
Name: environment, dtype: int64
1    62
0    12
Name: expansion, dtype: int64
0    73
1     1
Name: safety, dtype: int64
0    54
1    20
Name: modernize, dtype: int64
1    41
0    33
Name: integration, dtype: int64


In [45]:
#create a new df
data = [['environment', 31,43], ['expansion',62,12], ['safety', 1,73], ['modernize',20,54], ['integration',41,33]]
df_test = pd.DataFrame(data, columns = ['Goal', 'Projects_in_Category','Projects_NOT_in_Category'])
df_test['Percentage'] = ((100* df_test['Projects_in_Category']/(df_test['Projects_in_Category']).sum())).round(0)

In [46]:
df_test

,Goal,Projects_in_Category,Projects_NOT_in_Category,Percentage
0,environment,31,43,20.0
1,expansion,62,12,40.0
2,safety,1,73,1.0
3,modernize,20,54,13.0
4,integration,41,33,26.0


In [47]:
Project_Pie_Chart = alt.Chart(df_test).mark_arc().encode(
    theta=alt.Theta(field="Percentage", type="quantitative"),
    color=alt.Color(field="Goal", type="nominal",
                   scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BOLD_COLORS)
                   ),
)
Project_Pie_Chart

alt.Chart(...)

In [48]:
Project_Pie_Chart.save(f"./Charts/Project_Pie_Chart.png")

In [35]:
TIRCP_functions.basic_scatter_chart(df_test, 'Project_Category','Percentage','Project_Category') 

alt.Chart(...)

## 5. Projects with the most TIRCP funding with 0 recorded expenditures

In [36]:
df4 = df.loc[df['Progress'] == 'No Expenditures'] 

In [37]:
df5 = (df4.groupby(['Local_Agency','Project_Title',
                    'Award_Year']).agg({'TIRCP_Amount':'sum'}).reset_index().rename(columns = {'PPNO':'Count_of_Projects',
                                                                                               'TIRCP_Amount':'Total_TIRCP_Received', 'Project_Title':'Projects'}) 
      )

In [38]:
#Looking at projects with no expenditures 
df6 = df5.sort_values('Total_TIRCP_Received').tail(5)
df6

,Local_Agency,Projects,Award_Year,Total_TIRCP_Received
9,"LA County Metropolitan Transportation Authority, So Cal Regional Rail Authority (Metrolink)",Metrolink Antelope Valley Line Capital and Service Improvements,2020,"107,050,000.0"
1,Bay Area Rapid Transit District (BART),The Transbaby Corridor Core Capacity Program: Vehicle Acquisition,2020,"107,100,000.0"
2,Bay Area Rapid Transit District (BART),The Transbay Corridor Core Capacity Program: Vehicle Acquistion and Communications-Based Train Control System,2018,"318,600,000.0"
36,Santa Clara Valley Transportation Authority,"VTA’s BART Silicon Valley Extension, Phase II",2018,"730,000,000.0"
13,Los Angeles County Metropolitan Transportation Authority,Los Angeles Region Transit System Integration and Modernization Program of Projects,2018,"1,088,499,000.0"


In [39]:
projects_no_expenditures = TIRCP_functions.basic_bar_chart(df6, 'Projects','Total_TIRCP_Received','Projects') 
projects_no_expenditures

alt.Chart(...)

## 6. TIRCP funding by districts - 
* 8 projects do not have a district tagged.
* District-1 - Eureka
* District-2 - Redding
* District-3 - Marysville / Sacramento
* District-4 - Bay Area / Oakland
* District-5 - San Luis Obispo / Santa Barbara
* District-6 - Fresno / Bakersfield
* District-7 - Los Angeles
* District-8 - San Bernardino / Riverside
* District-9 - Bishop
* District-10 - Stockton
* District-11 - San Diego
* District-12 - Orange County

In [40]:
df.District.isna().sum()

8

In [41]:
df8 = df.groupby(['District']).agg({'PPNO':'count', 'TIRCP_Amount':'sum'}).rename(columns = {'PPNO':'Total_Projects', 'TIRCP_Amount':'Sum_of_TIRCP'}).reset_index()

In [42]:
df8['District'] = (df8['District'].replace({7:'Los Angeles',
                                            4:'Bay Area / Oakland',
                                            'VAR':'Various',
                                            10:'Stockton',
                                            11:'San Diego',
                                            3:'Marysville / Sacramento',
                                            12: 'Orange County',
                                            8: 'San Bernardino / Riverside',
                                            5:'San Luis Obispo / Santa Barbara',
                                            6:'Fresno / Bakersfield',
                                            1:'Eureka'
                                                 }))

In [43]:
df8 = df8.sort_values('Sum_of_TIRCP', ascending = False)
df8

,District,Total_Projects,Sum_of_TIRCP
5,Los Angeles,17,"2,458,538,000.0"
2,Bay Area / Oakland,14,"868,506,000.0"
10,Various,8,"690,852,000.0"
7,Stockton,4,"410,153,000.0"
8,San Diego,6,"175,947,000.0"
1,Marysville / Sacramento,5,"128,291,000.0"
9,Orange County,3,"58,937,000.0"
6,San Bernardino / Riverside,3,"54,204,000.0"
3,San Luis Obispo / Santa Barbara,3,"32,609,000.0"
4,Fresno / Bakersfield,2,"15,798,000.0"


In [44]:
District = TIRCP_functions.basic_bar_chart(df8, 'District','Sum_of_TIRCP','District') 
District

alt.Chart(...)